In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from scipy import stats

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# importando os dados
base = pd.read_csv('../input/aula-2-ia-dataset/CasasParaAlugar.csv', index_col = 0)
original = base
base.head()

In [ ]:
# substituindo quantitativos pela media 

areaMedia = base['area'].mean()
base.update(base['area'].fillna(areaMedia))

quartosMedia = base['rooms'].mean()
base.update(base['rooms'].fillna(quartosMedia))

banheiroMedia = base['bathroom'].mean()
base.update(base['bathroom'].fillna(banheiroMedia))

impostoMedia = base['property tax (R$)'].mean()
base.update(base['property tax (R$)'].fillna(impostoMedia))

valorMedia = base['rent amount (R$)'].mean()
base.update(base['rent amount (R$)'].fillna(valorMedia))

seguroMedia = base['fire insurance (R$)'].mean()
base.update(base['fire insurance (R$)'].fillna(seguroMedia))

taxasMedia = base['hoa (R$)'].mean()
base.update(base['hoa (R$)'].fillna(taxasMedia))

totalMedia = base['total (R$)'].mean()
base.update(base['total (R$)'].fillna(totalMedia))

totalMedia = base['parking spaces'].mean()
base.update(base['parking spaces'].fillna(totalMedia))

# verificando dados nulos/faltantes 
base.isnull().sum()

In [ ]:
# verificando dados qualitativos mais frequentes

print(base['city'].value_counts())
print('\n')
print(base['animal'].value_counts())
print('\n')
print(base['furniture'].value_counts())

In [ ]:
# substituindo dados qualitativos faltantes pelos mais frequentes
base['furniture'] = base['furniture'].fillna('not furnished')
base['animal'] = base['animal'].fillna('acept')
base['city'] = base['city'].fillna('São Paulo')

#excluindo os dados faltantes do andar 
#base = base.dropna(subset=['floor'])

# verificando dados nulos/faltantes 
base.isnull().sum()

In [ ]:
# arrumando valores de hoa (R$) total(R$)

novo = base['hoa (R$)'] < (20000)
base = base[novo]
novo = base['total (R$)'] < (20000)
base = base[novo]


In [ ]:
#Arrumando valores de rent amount (R$) e fire insurance (R$)
novo = base['rent amount (R$)'] < (17500)
base = base[novo]
novo = base['fire insurance (R$)'] < (300)
base = base[novo]

In [ ]:
# arrumando valores de property tax (R$)

novo = np.abs(stats.zscore(base['property tax (R$)'])) < 3
base = base[novo]


In [ ]:
# arrumando valores de area

novo = base['area'] < (5000)
base = base[novo]


In [ ]:
# arrumando valores de area

area = np.abs(stats.zscore(base['area'])) < 3
base = base[area]


In [ ]:
# feature engineering

# criando uma nova variavel que será a soma das taxas/impostos/seguro

base['sumtaxs'] = base['fire insurance (R$)']+ base['hoa (R$)'] + base['property tax (R$)']
base[['sumtaxs','fire insurance (R$)', 'hoa (R$)', 'property tax (R$)']]


In [ ]:
# notamos que a coluna 'rooms' foi considerada a menos importante.
# exclindo a coluna rooms

#base = base.drop(['rooms'], axis=1)


In [ ]:
# utilizando o metrodo MinMax para normalizar os valores entre 0 e 1
dados_quantitativos = ['area','bathroom','hoa (R$)','rent amount (R$)','property tax (R$)','fire insurance (R$)','parking spaces','total (R$)','sumtaxs','rooms']
scaler = MinMaxScaler()
data = base[dados_quantitativos]
scaler.fit(data)

data_scaled = scaler.transform(data)
data_scaled = pd.DataFrame(data_scaled)
data_scaled.columns = dados_quantitativos
data_scaled.index = base.index

base = base.drop(dados_quantitativos, axis=1)
base = pd.concat([base, data_scaled], axis=1)


In [ ]:
#trocando as colunas 'animals', 'furniture' por valores 0 e 1

base['furniture'].replace(to_replace='furnished', value=1, inplace=True)
base['furniture'].replace(to_replace='not furnished', value=0, inplace=True)

base['animal'].replace(to_replace='acept', value=1, inplace=True)
base['animal'].replace(to_replace='not acept', value=0, inplace=True)

In [ ]:
# OneHot Encoding
from sklearn.preprocessing import OneHotEncoder
base = base[base['city'].notna()]
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(base[['city']])
enc_df = pd.DataFrame(enc.transform(base[['city']]).toarray(), columns=enc.get_feature_names(['city']))
base.reset_index(drop=True, inplace=True)
enc_df.reset_index(drop=True, inplace=True)
base = base.join(enc_df)



In [ ]:
# excluindo as colunas 'city' e 'floor'
del base['floor']

In [ ]:
del base['city']
base

In [ ]:
# Pegamos o dataset df e separamos em x (entrada) e y (saida), numa separação 70% treino e 30% validação
from keras import *
from tensorflow.keras import regularizers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import plot_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

df = base

# Normalizamos os dados de df em uma escala de [0, 1]
# Estou fazendo isto aqui pois temos que "desnormalizar" na hora de gerar os gráficos de R²
column_names = base.columns
scaler = MinMaxScaler()
scaler.fit(df)
df = scaler.transform(df)
df = pd.DataFrame(df)
df.columns = column_names

# Pegamos o dataset df e separamos em x (entrada) e y (saida), numa separação 70% treino e 30% validação
input_dim = df.shape[1] - 1
x = df.drop(columns='total (R$)')
y = df['total (R$)']
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.30, random_state=42)

In [ ]:
NEURONIOS_CAMADA_INICIAL = 6
NEURONIOS_CAMADAS_INTERMEDIARIAS = [3]
USAR_DROPOUT = False
DROPOUT_VALUE = 0.2
TIPO_REGULARIZADOR = 'l1'
FN_ATIVACAO = 'tanh'

# #####################################################################################
# Definição da ARQUITETURA da Rede Neural
model = Sequential()

# Primeira camada da RNA (input_dim entradas)
model.add(Dense(units=NEURONIOS_CAMADA_INICIAL, input_dim=input_dim, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
# Camadas intermediárias
for UNITS in NEURONIOS_CAMADAS_INTERMEDIARIAS:
    model.add(Dense(units=UNITS, activation=FN_ATIVACAO, kernel_regularizer=TIPO_REGULARIZADOR))
    if USAR_DROPOUT:
        model.add(Dropout(DROPOUT_VALUE, input_shape=(120,)))
# Última camada da RNA (1 saída)
model.add(Dense(units=1, activation=FN_ATIVACAO))


#plot_model(model, show_shapes=True)

In [ ]:
from tensorflow import keras

CALLBACKS = [] # Definição dos callbacks a serem utilizados. Isso aqui é opcional, mas pode ajudar: https://keras.io/api/callbacks/early_stopping/
LOSS = 'mean_absolute_error' # 'mean_absolute_error', 'mean_squared_error'
BATCH_SIZE = 128
EPOCHS = 300
#OPTIMIZER = 'adam' # 'adam' é o mais utilizado. Caso prefira outro, como 'sgd', boa sorte!
OPTIMIZER = keras.optimizers.SGD(learning_rate=0.001)
# Compilação do modelo + Definição da Função de Loss e do Otimizador
model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=LOSS)

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=CALLBACKS,
    validation_data=(x_valid, y_valid),
)

In [ ]:
df_valid_scaled = np.column_stack((x_valid, y_valid))
df_valid = scaler.inverse_transform(df_valid_scaled)
y_true = y_valid


y_pred = model.predict(x_valid)
pred_df = pd.concat([pd.DataFrame(x_valid).reset_index(drop=True), pd.DataFrame(y_pred)], axis=1)
pred_df = scaler.inverse_transform(pred_df)
pred_df = pd.DataFrame(pred_df)
pred_df.columns = df.columns
pred_df

r2 = r2_score(y_true, y_pred)

fig, axes = pyplot.subplots(2, 1, figsize=(10, 10))

axes[0].plot(history.history['loss'])
axes[0].plot(history.history['val_loss'])
axes[0].set_title('model loss | "Quantidade de dados e colunas usadas: {}'.format(df.shape))
axes[0].set_ylabel('loss')
axes[0].set_xlabel('epoch')
axes[0].legend(['train', 'val'], loc='upper left')

axes[1].scatter(x=df_valid[:, -1], y=pred_df['total (R$)'])
# axes[0].plot(history.history['val_loss'])
axes[1].set_title('R² = {}'.format(r2))
axes[1].set_ylabel('y_pred')
axes[1].set_xlabel('y_true')

print("Quantidade de dados e colunas usadas: {}".format(df.shape))

pyplot.show()